In [ ]:
import pandas as pd
import numpy as np
import pytest
import uptide
import datetime
import pytz
from scipy.stats import linregress
from matplotlib.dates import date2num
import fnmatch
import os

In [ ]:
df1 = pd.read_fwf('data/1947ABE.txt', skiprows=9)

In [ ]:
df1.head(20)

In [ ]:
#df1 = pd.read_csv('data/aberdeen/2000ABE.txt', header=9, delim_whitespace=True)
# read in fixed-width file, skipping the header
df1 = pd.read_fwf('data/1947ABE.txt', skiprows=9)

# remove units row
df1.drop(0, inplace=True)

# rename sea level column
df1.rename(columns={df1.columns[3]: 'Sea Level'}, inplace=True)

# drop cycle column
df1.drop(columns=df1.columns[0], inplace=True)

# remove dodgy values
df1.replace(to_replace=".*T$",value={'Sea Level':np.nan},regex=True,inplace=True)
df1.replace(to_replace=".*N$",value={'Sea Level':np.nan},regex=True,inplace=True)
df1.replace(to_replace=".*M$",value={'Sea Level':np.nan},regex=True,inplace=True)
df1.replace(to_replace=".*T$",value={'Residual':np.nan},regex=True,inplace=True)
df1.replace(to_replace=".*N$",value={'Residual':np.nan},regex=True,inplace=True)
df1.replace(to_replace=".*M$",value={'Residual':np.nan},regex=True,inplace=True)

# convert strings to numbers and datetimes
df1['Date'] = pd.to_datetime(df1['Date'], format='%Y/%m/%d')
df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M:%S') # or pd.to_timedelta(df1['Time'], unit='s')
df1['Sea Level'] = df1['Sea Level'].astype('float')
df1['Residual'] = df1['Residual'].astype('float')

In [ ]:
type(df1['datetime'].iloc[0])

In [ ]:
type(df1.index)

In [ ]:
def read_tidal_data(filename):
   # Reads in filename into a pandas dataframe which is cleaned and formatted. Returns the dataframe.
   
   # read in fixed-width file, skipping the header
   df = pd.read_fwf(filename, skiprows=9)

   # remove units row
   df.drop(0, inplace=True)

   # rename sea level column
   df.rename(columns={df.columns[3]: 'Sea Level'}, inplace=True)

   # drop cycle column
   df.drop(columns=df.columns[0], inplace=True)

   # remove dodgy values
   df.replace(to_replace=".*T$",value={'Sea Level':np.nan},regex=True,inplace=True)
   df.replace(to_replace=".*N$",value={'Sea Level':np.nan},regex=True,inplace=True)
   df.replace(to_replace=".*M$",value={'Sea Level':np.nan},regex=True,inplace=True)
   df.replace(to_replace=".*T$",value={'Residual':np.nan},regex=True,inplace=True)
   df.replace(to_replace=".*N$",value={'Residual':np.nan},regex=True,inplace=True)
   df.replace(to_replace=".*M$",value={'Residual':np.nan},regex=True,inplace=True)

   # convert strings to numbers and datetimes
   df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
   df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S') # or pd.to_timedelta(df['Time'], unit='s')
   df['Sea Level'] = df['Sea Level'].astype('float')
   df['Residual'] = df['Residual'].astype('float')

   # create datetime column
   df['datetime'] = df['Date'] + pd.to_timedelta(df['Time'].dt.time.astype(str))

   # set datetime as index
   df.set_index('datetime', inplace=True)

   # NB don't drop Date and Time columns as they are used in tests

   return df

In [ ]:
data = read_tidal_data("data/1947ABE.txt")

In [ ]:
data.head(10)

In [ ]:
assert "Sea Level" in data.columns
assert type(data.index) == pd.core.indexes.datetimes.DatetimeIndex
assert data['Sea Level'].size == 8760
assert '1947-01-01 00:00:00' in data.index
assert '1947-12-31 23:00:00' in data.index

# check for M, N and T data; should be NaN
assert data['Sea Level'].isnull().any()
assert pd.api.types.is_float_dtype(data['Sea Level'])

I believe next step is to use uptide module to work out tidal constuents but first have to remove np.nan values. Code that illustrates how to use the uptide module is found in the website: https://jhill1.github.io/SEPwC.github.io/tides_python.html so just have to follow that.

Not sure if I need to refactor join_data so that it can handle more than two files - actually no because the skeleton function has arguments data1 and data2 only.

In [ ]:
def join_data(data1, data2):
   # Joins data1 and data2 vertically and returns the resulting dataframe
   
   # concatenate the data
   combined_data = pd.concat([data1, data2])
   # sort the index by ascending datetime
   combined_data.sort_index(inplace=True)

   return combined_data

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']

data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

In [ ]:
data

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']

data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

assert "Sea Level" in data.columns
assert type(data.index) == pd.core.indexes.datetimes.DatetimeIndex
assert data['Sea Level'].size == 8760*2

# check sorting (we join 1947 to 1946, but expect 1946 to 1947)
assert data.index[0] == pd.Timestamp('1946-01-01 00:00:00')
assert data.index[-1] == pd.Timestamp('1947-12-31 23:00:00')

# check you get a fail if two incompatible dfs are given
data2.drop(columns=["Sea Level","Time"], inplace=True)
data = join_data(data1, data2)

In [ ]:
data

In [ ]:
def extract_single_year_remove_mean(year, data):
   # Takes in dataframe data containing multiple years of data and returns a dataframe containing only data from the year year. 
   # Also removes the mean by normalising the data.

   if int(year) in data.index.year:
      data = data[data.index.year == int(year)] # ensure year is an int and not a string
   else:
      print("The data does not contain the given year!")
      return -1

   # Normalise sea level to remove the mean as in mini course
   data['Sea Level'] = data['Sea Level'] - data['Sea Level'].mean()
    
   return data

## Not sure if I have to normalise Residual too?? As idk if this will affect the coefficient calculations.

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']

data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

year1947 = extract_single_year_remove_mean("1947",data)
assert "Sea Level" in year1947.columns
assert type(year1947.index) == pd.core.indexes.datetimes.DatetimeIndex
assert year1947['Sea Level'].size == 8760

mean = np.mean(year1947['Sea Level'])
print(mean)
# check mean is near zero
assert mean == pytest.approx(0)

# check something sensible when a year is given that doesn't exist

In [ ]:
def extract_section_remove_mean(start, end, data):
    # Takes in dataframe data and returns all rows between start date and end date inclusive.
    # Also removes the mean by normalising the data.

    # Ensure the index is a DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        data = data.copy()
        data.index = pd.to_datetime(data.index)
    
    # Convert arguments to datetimes
    start = pd.to_datetime(start).date()
    end = pd.to_datetime(end).date()
    
    # Filter the DataFrame
    data = data[(data.index.date >= start) & (data.index.date <= end)]

    # Normalise sea level to remove the mean as in mini course
    data['Sea Level'] = data['Sea Level'] - data['Sea Level'].mean()

    return data

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']

data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

year1946_47 = extract_section_remove_mean("19461215", "19470310", data)

In [ ]:
year1946_47

In [ ]:
assert "Sea Level" in year1946_47.columns
assert type(year1946_47.index) == pd.core.indexes.datetimes.DatetimeIndex
assert year1946_47['Sea Level'].size == 2064

mean = np.mean(year1946_47['Sea Level'])
# check mean is near zero
assert mean == pytest.approx(0)

data_segment = extract_section_remove_mean("19470115", "19470310", data1)
assert "Sea Level" in data_segment.columns
assert type(data_segment.index) == pd.core.indexes.datetimes.DatetimeIndex
assert data_segment['Sea Level'].size == 1320

mean = np.mean(data_segment['Sea Level'])
# check mean is near zero
assert mean == pytest.approx(0)

# check something sensible is done when dates are formatted correctly.

In [ ]:
def tidal_analysis(data, constituents, start_datetime):
   # Returns amplitude and phase for given constituents and sea level data using uptide library

   # Need to remove NaNs before calculating tidal constituents
   data = data[~data['Sea Level'].isna()]

   # we create a Tides object with a list of the consituents we want.
   tide = uptide.Tides(constituents)

   # We then set out start time. All data must then be in second since this time
   tide.set_initial_time(start_datetime)

   # calculate seconds since start_datetime
   seconds_since = (data.index.astype('int64').to_numpy()/1e9) - start_datetime.timestamp()

   # Calculate amplitude and phase
   amp, pha = uptide.harmonic_analysis(tide, data['Sea Level'].to_numpy(), seconds_since)
   
   return amp, pha

In [ ]:
data_segment =extract_section_remove_mean("19460115", "19470310", data)

In [ ]:
data_segment.isna().sum()

In [ ]:
data_segment = data_segment[~data_segment['Sea Level'].isna()]

In [ ]:
data_segment.isna().sum()

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']
data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

data_segment =extract_section_remove_mean("19460115", "19470310", data)

constituents  = ['M2', 'S2']
tz = pytz.timezone("utc")
start_datetime = datetime.datetime(1946,1,15,0,0,0, tzinfo=tz)
amp,pha = tidal_analysis(data_segment, constituents, start_datetime)
print(amp, pha)
# for Aberdeen, the M2 and S2 amps are 1.307 and 0.441
assert amp[0] == pytest.approx(1.307,abs=0.1)
assert amp[1] == pytest.approx(0.441,abs=0.1)

In [ ]:
FD = pd.read_csv('h333.csv', header=None)

In [ ]:
FD

## Only other idea I have is to calculate slope for each day in the dataset and then average over all values to get the whole year. Would have to average the p value as well so doesn't really make sense.

In [ ]:
def sea_level_rise(data):
   # Uses linear regression to calculate sea level rise in metres per year. Returns the slope (sea level rise) and p value.

   # Remove NaNs
   data = data[~data['Sea Level'].isna()]
   
   # Convert dates to numbers for the regression
   x = date2num(data.index)
   y = data['Sea Level']
   
   # use scipy.stats.linregress
   slope, intercept, r, p, se = linregress(x, y)

   return slope, p

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']
data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

slope, p_value = sea_level_rise(data)
print(slope, p_value)
assert slope == pytest.approx(2.94e-05,abs=1e-7)
assert p_value == pytest.approx(0.427,abs=0.1)

In [ ]:
2.8479573538948342e-05+1e-7

In [ ]:
def find_longest_contiguous_data(data):
    # Returns the start date and end date of the longest contiguous period of data (no missing timestamps or values) as well as the length of this period.

    # Ensure the index is sorted
    data = data.sort_index()
    
    # Create a boolean mask for valid rows (no NaNs and hourly frequency maintained)
    # Step 1: Identify where the data is valid (not NaN)
    valid = data['Sea Level'].notna()
    #print(valid)
    # Step 2: Identify time gaps larger than 1 hour
    time_diffs = data.index.to_series().diff().gt(pd.Timedelta(hours=1))
    time_diffs.iloc[0] = False  # First entry has no diff
    #print(time_diffs)
    # Combine both masks: invalid if NaN or gap in time
    invalid = ~valid | time_diffs
    #print(invalid)
    # Assign a group number that increases whenever a break (invalid) occurs
    group_id = (~invalid).cumsum() * (~invalid)
    #print(group_id)
    # Count the size of each valid group
    group_sizes = group_id[group_id > 0].value_counts()
    print(group_sizes.max())
    if group_sizes.empty:
        return None, None, 0  # No valid data

    longest_group = group_sizes.idxmax()
    max_length = group_sizes.max()

    # Get timestamps of the longest group
    period_data = data[group_id == longest_group]
    start = period_data.index[0]
    end = period_data.index[-1]

    return start, end, max_length

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']
data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

st, end, length = find_longest_contiguous_data(data)
print(st, end, length)

In [ ]:
data

In [ ]:
def get_longest_contiguous_data(data):
    # Returns the start date and end date of the longest contiguous period of data (no missing timestamps or values) as well as the length of this period.

    # Ensure the index is sorted and is a DatetimeIndex
    data = data.sort_index()
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")

    # Step 1: Identify invalid rows (NaN or time gaps)
    valid = data['Sea Level'].notna()
    time_gap = data.index.to_series().diff().gt(pd.Timedelta(hours=1))
    time_gap.iloc[0] = False
    breaks = (~valid) | time_gap

    # Step 2: Mark groups - a new group starts after a break
    group_id = breaks.cumsum()

    # Step 3: Group by group_id and find the longest one
    data_valid = data[~breaks]
    if data_valid.empty:
        return None, None, 0

    groups = data_valid.groupby(group_id)
    longest_group = max(groups, key=lambda g: len(g[1]))

    start = longest_group[1].index[0]
    end = longest_group[1].index[-1]
    length = len(longest_group[1])

    return start, end, length

In [ ]:
gauge_files = ['data/1946ABE.txt', 'data/1947ABE.txt']
data1 = read_tidal_data(gauge_files[1])
data2 = read_tidal_data(gauge_files[0])
data = join_data(data1, data2)

st, end, length = find_longest_contiguous_data(data)
print(st, end, length)

In [ ]:
def load_folder(folder_name):
    # Reads in all .txt files in folder_name and joins them into one text file. This file is returned.

    # Get all txt files in the folder
    txt_files = [f for f in os.listdir(folder_name) if fnmatch.fnmatch(f, '*.txt')]

    data = read_tidal_data(folder_name + '/' + txt_files[0])
    # Concatenate all the txt files into one
    for file in txt_files[1:]:
        data_tmp = read_tidal_data(folder_name + '/' + file)
        data = join_data(data, data_tmp)

    return data

In [ ]:
data = load_folder('data/whitby')

print(len(data))
print(data.head())

In [ ]:
print(data.info())

### "The program should print the tidal data, the sea-level rise and the longest contiguous period of data (i.e. without any missing data) from the data loaded."

In [ ]:
data.index.duplicated().sum()

In [ ]:
data = data[~data.index.duplicated(keep='first')]

In [ ]:
st, end, length = get_longest_contiguous_data(data)
print("Start date: ", st, "End date: ", end, "Longest contiguouos period: ", length)

slope, p_value = sea_level_rise(data)
print("Sea level rise rate (slope in metres per unit time): ", slope, "p-value: ", p)

constituents  = ['M2', 'S2']
start_datetime = data.index[0]
amp, pha = tidal_analysis(data, constituents, start_datetime)
print("Amplitude of M2, S2: ", amp)